In [1]:
%cd ../

/Users/hoangle/Uni/Thesis


In [2]:
import torch
from recbole.data import (
    create_dataset,
    data_preparation,
)
from recbole.utils import (
    get_model,
    get_trainer,
    init_seed,
)

from src.real_temporal import TimeCutoffDataset

# Load checkpoint

In [3]:
path = "/Users/hoangle/Desktop/BERT4Rec-Jul-24-2024_23-20-36.pth"

# Declare necessary things

In [4]:
# config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file=path)
chkpt = torch.load(path, map_location=torch.device('cpu'))

config = chkpt['config']
config['device'] = torch.device('cpu')
config['use_gpu'] = False

init_seed(config["seed"], config["reproducibility"])

if config["use_cutoff"] is True:
    dataset = TimeCutoffDataset(config)
else:
    dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Define model
model_name = config["model"]
model = get_model(model_name)(config, train_data._dataset).to(config["device"])
model.load_state_dict(chkpt['state_dict'])
model.load_other_parameter(chkpt['other_parameter'])

# Define trainer
trainer = get_trainer(config["MODEL_TYPE"], config["model"])(config, model)

# Start eval with test data

In [5]:
trainer.evaluate(test_data, model_file=path)

OrderedDict([('ndcg@10', 0.0011),
             ('precision@10', 0.0002),
             ('recall@10', 0.0024),
             ('mrr@10', 0.0007),
             ('hit@10', 0.0024),
             ('map@10', 0.0007)])